## Importar librerias y configurar parámetros de base de datos y ruta de archivos

In [4]:
# libreria para leer .docx
from docx import Document
# libreria para expresiones regulares
import re
## SQL Server library
import pyodbc
sql_conn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER=CO2038696W2\SQLEXPRESS;DATABASE=ComoNuevos;Trusted_Connection=yes') 
#Dataframes
import pandas as pd
#para comparar textos
from fuzzywuzzy import fuzz
#Para listar los archivos de directorios
from os import listdir
mypath = "C:/Users/ivan.gomez.r/EY/Natural Language Processing/contracts/"

## Funciones 

In [5]:
#Recibe la ruta de un archivo, lo lee y concatena todos los parrafos en una sola variable "contract"
def leerArchivo(file_name):    
    f = open(file_name, 'rb')
    document = Document(f)
    docVector = []
    contract=""
    for p in document.paragraphs:
        contract = contract +  "\n" + p.text
    return contract


#Recibe una variable "contract" que representa un documento, y mediante expresiones regulares extrae los datos 
#y los inserta en la tala de base de datos
def contratosRegEx(documento, file_name, score_dist, score_bow):
    #Extraeer clase
    searchResult = re.search(r"CLASE:.*?[\n|\t]", documento).group()
    clase = searchResult[7:len(searchResult)-1].upper()
    #Extraer marca
    searchResult = re.search(r"MARCA:.*?[\n|\t]", documento).group()
    marca = searchResult[7:len(searchResult)-1].upper()
    #Extraer modelo
    searchResult = re.search(r"MODELO:.*?[\n|\t]", documento).group()
    modelo = searchResult[8:len(searchResult)-1].upper()
    #Extraer línea
    searchResult = re.search(r"LÍNEA:.*?[\n|\t]", documento).group()
    linea = searchResult[7:len(searchResult)-1].upper()
    #Extraer placa
    searchResult = re.search(r"PLACA:.*?[\n|\t]", documento).group()
    placa = searchResult[7:len(searchResult)-1].upper()
    #Extraer color
    searchResult = re.search(r"COLOR:.*?[\n|\t]", documento).group()
    color = searchResult[7:len(searchResult)-1].upper()
    #Extraer puertas
    searchResult = re.search(r"PUERTAS:.*?[\n|\t]", documento).group()
    puertas = searchResult[9:len(searchResult)-1].upper()
    #Extraer potencia
    searchResult = re.search(r"POTENCIA HP:.*?[\n|\t]", documento).group()
    potencia = searchResult[13:len(searchResult)-1].upper()
    #Extraer num_motor
    searchResult = re.search(r"MOTOR:.*?[\n|\t]", documento).group()
    num_motor = searchResult[8:len(searchResult)-1].upper()
    #Extraer precio
    searchResult = re.search(r"Precio.*?\)", documento).group()
    searchResult = re.search(r"\(.*?\)", searchResult).group()
    precio = searchResult[3:len(searchResult)-1].replace('.','')
    precio

    #Conectar e insertar en base de datos
    cursor = sql_conn.cursor()
    query = "INSERT INTO Contratos VALUES ('%s','%s', '%s','%s', '%s','%s', '%s','%s', '%s','%s','%s','%s','%s')" % (clase, marca, modelo, linea, placa, color, puertas, potencia, num_motor, precio, file_name, score_dist, score_bow)
    cursor.execute(query)
    sql_conn.commit()
    

## Recorrer todos los archivos del directorio y evaluar coincidencia

In [7]:
files = listdir(mypath)
fileRef = leerArchivo(mypath+"I30_MKS126.docx")

docVector = []
for i in range(len(files)):
    if (files[i][-4:] == 'docx' ):
        fileTem = leerArchivo(mypath + files[i])
        score_dist_tem = fuzz.ratio(fileRef,fileTem)
        print ('Para el archivo: %s Similitud: %s' % (files[i], score_dist_tem) )
        docVector.append(fileTem)
        if (score_dist_tem>40):
            contratosRegEx(fileTem, files[i] ,score_dist_tem, 0)    

Para el archivo: CERATO_PRO_.docx Similitud: 49
Para el archivo: compra_396.docx Similitud: 81
Para el archivo: CONTRATO DE MANDATO.docx Similitud: 10
Para el archivo: CONTRATO DE MANDATO_DW139.docx Similitud: 10
Para el archivo: contrato_compra_DXM627.docx Similitud: 49
Para el archivo: Fiesta.docx Similitud: 80
Para el archivo: FOCUS_HBU426.docx Similitud: 81
Para el archivo: I25_ZYY547.docx Similitud: 50
Para el archivo: I30_MKS126.docx Similitud: 100
Para el archivo: MAZDA _contrato_compra_NDW139.docx Similitud: 86
Para el archivo: Project Charter.docx Similitud: 1
Para el archivo: Tarea1.docx Similitud: 0


In [26]:
print(index[tfidf[corpus[8]]])

[0.44328287 0.3703545  0.0358165  0.0358165  0.40674382 0.43130946
 0.42612386 0.42286518 0.99999976 0.51067173 0.00232538 0.00465207]


## Cargar tabla SQL Server como un dataframe

In [4]:
contratos = pd.read_sql("SELECT * FROM Contratos", sql_conn)
contratos

,CLASE,MARCA,MODELO,LINEA,PLACA,COLOR,PUERTAS,POTENCIA,NUM_MOTOR,PRECIO,ARCHIVO,SCORE_DIST,SCORE_BOW
0,AUTOMÓVIL,KIA,2015,CERATO PRO,IFT732,PLATA,4,130,CVSD147AA99,39500000,CERATO_PRO_.docx,49,0
1,AUTOMÓVIL,MAZDA,2015,MAZDA 2,ZZR396,ROJO EUROPA,5,104,AUR768HG4545,33000000,compra_396.docx,81,0
2,AUTOMÓVIL,HYUNDAI,2017,ACCENT,DXM627,BLANCO PURO,4,122,4LCEU209965,41250000,contrato_compra_DXM627.docx,49,0
3,AUTOMÓVIL,FORD,2016,FIESTA TITANIUM,INL443,PLATA,4,120,FF396AAAR093049,37000000,Fiesta.docx,80,0
4,AUTOMÓVIL,FORD,2013,FOCUS SE AT,HBU426,GRIS NOCTURNO,4,160,L147675,33500000,FOCUS_HBU426.docx,81,0
5,AUTOMÓVIL,HYUNDAI,2015,ACCENT GL,ZYY547,GRIS MEDIO,4,107,4LCEU208675,32250000,I25_ZYY547.docx,50,0
6,AUTOMÓVIL,HYUNDAI,2013,I 30 GLS,MKS126,PLATA CONTINENTAL,5,121,G4FGCU528020,36750000,I30_MKS126.docx,100,0
7,AUTOMÓVIL,KIA,2013,RIO UB BX,NDW139,GRIS,4,109,G4FACS434762,28700000,MAZDA _contrato_compra_NDW139.docx,86,0
8,AUTOMÓVIL,KIA,2015,CERATO PRO,IFT732,PLATA,4,130,CVSD147AA99,39500000,CERATO_PRO_.docx,49,0
9,AUTOMÓVIL,MAZDA,2015,MAZDA 2,ZZR396,ROJO EUROPA,5,104,AUR768HG4545,33000000,compra_396.docx,81,0


### Modelo calculando la similitud de palabrás principales "Bag of Words"

- Paso 1: Remover palabras conectoras, pronombre y otros
- Paso 2: Remover palabras que solo aparecen una vez
- Paso 3: Creació del diccionario con las palabras principales de cada documento
- Paso 4: Creación del corpus para cada documento basado en el diccionario de datos construido
- Paso 5: Usar funcion de similaridad basado en "Cosine similarity"

Cosine similarity is a measure of similarity between two non-zero vectors of an inner product space that measures the cosine of the angle between them. The cosine of 0° is 1, and it is less than 1 for any angle in the interval (0,π] radians. It is thus a judgment of orientation and not magnitude: two vectors with the same orientation have a cosine similarity of 1, two vectors oriented at 90° relative to each other have a similarity of 0, and two vectors diametrically opposed have a similarity of -1, independent of their magnitude.  source: wikipedia

In [5]:
#docVector[7]

from pprint import pprint  # pretty-printer
from collections import defaultdict
#from gensim.corpora import Dictionary
import gensim as g

# remove common words and tokenize
stoplist = set('en las la los lo un uno que y con se de no a por al es esas el mas más'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
          for document in docVector]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]
         for text in texts]

dictionary = g.corpora.Dictionary(texts)
dictionary.save('/temp/diccionario.dict')  # store the dictionary, for future reference
print(dictionary)
print(dictionary.token2id)


C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Dictionary(353 unique tokens: ['($', '(1)', '(12)', '(2)', '(8)']...)
{'($': 0, '(1)': 1, '(12)': 2, '(2)': 3, '(8)': 4, '1.000.000).': 5, '1.012.343.635': 6, '1.750.000),': 7, '1fadp3f27dl147675': 8, '2015': 9, '2018.': 10, '31.750.000),': 11, '4': 12, '5': 13, '80.167.459,': 14, '_________________': 15, '___________________': 16, 'acordado': 17, 'acta': 18, 'acudir': 19, 'adelante': 20, 'afecte': 21, 'agotará': 22, 'alternativo': 23, 'ante': 24, 'anterior': 25, 'antes': 26, 'arbitraje': 27, 'arbitramento': 28, 'arreglo': 29, 'automotor': 30, 'automóvil': 31, 'año': 32, 'bien': 33, 'bogotá': 34, 'bogotá,': 35, 'buen': 36, 'c.c.': 37, 'capacidad:': 38, 'cedula': 39, 'celebrar': 40, 'centro': 41, 'chasis:': 42, 'cincuenta': 43, 'circunstancia': 44, 'ciudad': 45, 'ciudadanía': 46, 'civiles': 47, 'clase:': 48, 'cláusula': 49, 'cláusulas:': 50, 'color:': 51, 'comerciales': 52, 'comercio': 53, 'como': 54, 'comprador': 55, 'comprador,': 56, 'compraventa': 57, 'compromisoria:': 58, 'conciliac

In [6]:
corpus = [dictionary.doc2bow(text) for text in texts]
g.corpora.MmCorpus.serialize('/temp/diccionario.dict', corpus)  # store to disk, for later use
print(corpus)

[[(0, 4), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 2), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 2), (31, 1), (32, 1), (33, 1), (34, 2), (35, 2), (36, 1), (37, 3), (38, 1), (39, 2), (40, 1), (41, 1), (42, 1), (43, 2), (44, 1), (45, 1), (46, 2), (47, 1), (48, 1), (49, 3), (50, 1), (51, 1), (52, 1), (53, 1), (54, 3), (55, 3), (56, 1), (57, 3), (58, 1), (59, 1), (60, 1), (61, 3), (62, 2), (63, 1), (64, 1), (65, 1), (66, 1), (67, 2), (68, 1), (69, 2), (70, 1), (71, 9), (72, 1), (73, 2), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 2), (80, 1), (81, 1), (82, 1), (83, 2), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 2), (93, 1), (94, 1), (95, 4), (96, 1), (97, 1), (98, 1), (99, 1), (100, 3), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 2), (107, 1), (108, 1), (109, 1), (110, 1)

In [7]:
tfidf = g.models.TfidfModel(corpus)
lsi = g.models.LsiModel(corpus, id2word=dictionary, num_topics=2)
index = g.similarities.SparseMatrixSimilarity(tfidf[corpus], num_features=353)

In [25]:
#results = pd.DataFrame(0, index=len(docVector), columns=len(docVector))

print(docVector[8])

#for i in range(0, len(docVector)):
print(index[tfidf[corpus[8]]])



CONTRATO DE COMPRAVENTA DE VEHÍCULO AUTOMOTOR

El señor IVAN DARIO GOMEZ RINCON, mayor de edad, domiciliado en Bogotá, identificado con cedula de ciudadanía No.1.012.343.635, quien en adelante se denominará EL COMPRADOR, y JOSÉ OMAR FRANCO BAUTISTA, mayor de edad, domiciliada en Bogotá, identificada con cedula de ciudadanía No. 19.050.527 y en adelante se denominará EL VENDEDOR, hemos acordado celebrar contrato de compraventa que se regirá por las normas civiles y comerciales que regulan la materia, según las siguientes cláusulas:
Primera. Objeto: EL VENDEDOR transfiere a título de venta a EL COMPRADOR la propiedad del vehículo automotor que a continuación se identifica:
CLASE: Automóvil	MARCA: HYUNDAI	MODELO: 2013	 	LÍNEA: i 30 GLS
PLACA: MKS126		COLOR: PLATA CONTINENTAL	CAPACIDAD: 5 psj MATRICULADO EN: Bogotá
PUERTAS: 5 		POTENCIA HP: 121
NÚMERO DE MOTOR:  G4FGCU528020
NÚMERO DE SERIE: KMHD351BADU010226 
NÚMERO DE CHASIS: KMHD351BADU010226
Segunda. Precio: Las partes pactan la suma 

In [9]:
sql_conn.close()

f = open(file_name, 'rb')
document = Document(f)
docVector = []
for p in document.paragraphs:
    docVector.append(p.text)

tabsPart = set(docVector[4].split('\t'))
tabsPart
docVector[4]

sequence = ""
for i in range(len(docVector)):
    sequence = sequence + "\n" + docVector[i]   
sequence

def defineVar(file_name):    
    f = open(file_name, 'rb')
    document = Document(f)
    docVector = []
    contract=""
    for p in document.paragraphs:
        contract = contract +  "\n" + p.text
    #print (contract)
    contratosETL(contract)

NameError: name 'file_name' is not defined